In [2]:
import json
from collections import Counter

In [2]:
with open('prop2constraints.json', 'r') as f:
    constraint_dict = json.load(f)

In [3]:
constraint_dict

{'P9591': {'subject type constraint': {'P2309': ['Q21503252'],
   'P2308': ['Q811979']},
  'distinct-values constraint': {},
  'single-value constraint': {},
  'format constraint': {'P1793': ['[1-9][0-9]*']},
  'property scope constraint': {'P5314': ['Q54828448']},
  'allowed-entity-types constraint': {'P2305': ['Q29934200']},
  'item-requires-statement constraint': {'P2306': ['Property:P149'],
   'P2305': ['Q994776']}},
 'P7111': {'subject type constraint': {'P2309': ['Q21503252'],
   'P2308': ['Q5']},
  'single-value constraint': {},
  'distinct-values constraint': {},
  'format constraint': {'P1793': ['([a-z]+/players|[a-z0-9]+)/[0-9]+'],
   'P2316': ['Q62026391']},
  'item-requires-statement constraint': {'P2306': ['Property:P106'],
   'P2316': ['Q21502408']},
  'allowed-entity-types constraint': {'P2305': ['Q29934200']},
  'property scope constraint': {'P5314': ['Q54828448', 'Q54828450']}},
 'P10678': {'subject type constraint': {'P2309': ['Q21503252'],
   'P2308': ['Q5']},
  'sin

In [4]:
relation_mapping_filename="relation_mapping.json"

with open(relation_mapping_filename, "r") as f:
    id2relation = json.load(f)

In [5]:
len(id2relation)

1187

In [6]:
new_constraint_dict = {}

for rel in id2relation:
    if rel in constraint_dict.keys():
        new_constraint_dict[rel] = constraint_dict[rel]

len(new_constraint_dict)


1180

In [7]:
del constraint_dict

In [8]:
new_constraint_dict

{'P2673': {'inverse constraint': {'P2306': ['Property:P2674'],
   'P2316': ['Q21502408']},
  'allowed-entity-types constraint': {'P2305': ['Q29934200'],
   'P2316': ['Q21502408']},
  'contemporary constraint': {'P2316': ['Q62026391']},
  'property scope constraint': {'P5314': ['Q54828448'],
   'P2316': ['Q62026391']},
  'subject type constraint': {'P2308': ['Q12280',
    'Q44377',
    'Q12323',
    'Q18984099'],
   'P2309': ['Q21503252']},
  'value-type constraint': {'P2308': ['Q12280',
    'Q44377',
    'Q12323',
    'Q18984099'],
   'P2309': ['Q21503252']},
  'value-requires-statement constraint': {'P2306': ['Property:P17']}},
 'P571': {'single-best-value constraint': {'P4155': ['Property:P518',
    'Property:P1001',
    'Property:P1810',
    'Property:P2868',
    'Property:P4649',
    'Property:P1545',
    'Property:P5102',
    'Property:P3831',
    'Property:P5168',
    'Property:P8338',
    'Property:P1013',
    'Property:P1365']},
  'conflicts-with constraint': {'P2306': ['Proper

In [9]:
c = Counter()

In [10]:
for rel in new_constraint_dict:
    for key in new_constraint_dict[rel]:
        c[key] += 1

In [11]:
c

Counter({'inverse constraint': 83,
         'allowed-entity-types constraint': 852,
         'contemporary constraint': 127,
         'property scope constraint': 1016,
         'subject type constraint': 842,
         'value-type constraint': 671,
         'value-requires-statement constraint': 159,
         'single-best-value constraint': 53,
         'conflicts-with constraint': 265,
         'allowed qualifiers constraint': 260,
         'none-of constraint': 108,
         'one-of constraint': 72,
         'range constraint': 153,
         'allowed units constraint': 184,
         'integer constraint': 77,
         'no-bounds constraint': 43,
         'single-value constraint': 112,
         'item-requires-statement constraint': 257,
         'citation needed constraint': 129,
         'difference-within-range constraint': 12,
         'required qualifier constraint': 89,
         'multi-value constraint': 8,
         'distinct-values constraint': 36,
         'symmetric constraint

In [12]:
missed_scope_props = []

for prop in new_constraint_dict:
    if ("property scope constraint" in new_constraint_dict[prop] and \
    not ("P2305" in new_constraint_dict[prop]["property scope constraint"] or "P5314" in new_constraint_dict[prop]["property scope constraint"])):
        print(prop)
        missed_scope_props.append(prop)

for prop in missed_scope_props:
    del new_constraint_dict[prop]["property scope constraint"]
len(new_constraint_dict)            

P7589
P5230


1180

In [13]:
filtered_properties = {}

for prop in new_constraint_dict:
    if "property scope constraint" in new_constraint_dict[prop]:
        if "P2305" in new_constraint_dict[prop]["property scope constraint"]:
            if "Q54828448" in new_constraint_dict[prop]["property scope constraint"]["P2305"]:
                filtered_properties[prop] = new_constraint_dict[prop]
               
        else:
            if "Q54828448" in new_constraint_dict[prop]["property scope constraint"]["P5314"]:
                filtered_properties[prop] = new_constraint_dict[prop]

    else:
        filtered_properties[prop] = new_constraint_dict[prop]

len(filtered_properties)

1142

In [14]:
missed_allowed_entity_types = []

for prop in filtered_properties:
    if "allowed-entity-types constraint" in new_constraint_dict[prop]:
        if "P2305"  not in filtered_properties[prop]["allowed-entity-types constraint"].keys():
            print(prop)
            missed_allowed_entity_types.append(prop)
            
for prop in missed_allowed_entity_types:
    del filtered_properties[prop]["allowed-entity-types constraint"]

len(filtered_properties)

P5230
P3080


1142

In [15]:
filtered_properties_ = {}

for prop in filtered_properties:
    if "allowed-entity-types constraint" in new_constraint_dict[prop]:
        
        if "Q29934200" in filtered_properties[prop]["allowed-entity-types constraint"]["P2305"]:
            filtered_properties_[prop] = new_constraint_dict[prop]
    else:
        filtered_properties_[prop] = new_constraint_dict[prop]
        
filtered_properties = filtered_properties_
len(filtered_properties)

1141

In [16]:
data_constraints = ["subject type constraint", "value-type constraint" ]
                            # "allowed-entity-types constraint", 
                            #   "one-of constraint"]

In [17]:
properties_with_data_constraints = {}
for prop in filtered_properties:

    for constraint in data_constraints:
        if constraint in filtered_properties[prop]:
            if 'P2308' in filtered_properties[prop][constraint] and 'P2309' in filtered_properties[prop][constraint]:
                properties_with_data_constraints[prop] = new_constraint_dict[prop]
            else:
                if  prop in properties_with_data_constraints:
                    del new_constraint_dict[prop][constraint]
                else:
                    del filtered_properties[prop][constraint]
                

In [18]:
len(properties_with_data_constraints)

949

In [19]:
properties_with_data_constraints

{'P2673': {'inverse constraint': {'P2306': ['Property:P2674'],
   'P2316': ['Q21502408']},
  'allowed-entity-types constraint': {'P2305': ['Q29934200'],
   'P2316': ['Q21502408']},
  'contemporary constraint': {'P2316': ['Q62026391']},
  'property scope constraint': {'P5314': ['Q54828448'],
   'P2316': ['Q62026391']},
  'subject type constraint': {'P2308': ['Q12280',
    'Q44377',
    'Q12323',
    'Q18984099'],
   'P2309': ['Q21503252']},
  'value-type constraint': {'P2308': ['Q12280',
    'Q44377',
    'Q12323',
    'Q18984099'],
   'P2309': ['Q21503252']},
  'value-requires-statement constraint': {'P2306': ['Property:P17']}},
 'P186': {'value-type constraint': {'P2308': ['Q214609',
    'Q2095',
    'Q11344',
    'Q4936952',
    'Q25403900',
    'Q1362373',
    'Q7239',
    'Q40050',
    'Q223557',
    'Q15053464',
    'Q206615',
    'Q37756',
    'Q11173',
    'Q31839438',
    'Q10683158',
    'Q19829125'],
   'P2309': ['Q30208840'],
   'P2303': ['Q113796']},
  'allowed qualifiers c

In [20]:
prop2constraint = {}
for prop in properties_with_data_constraints:
    const_dict = {}
    if 'subject type constraint' in properties_with_data_constraints[prop]:
        const_dict['subject type constraint'] = properties_with_data_constraints[prop]['subject type constraint']
    if 'value-type constraint' in properties_with_data_constraints[prop]: 
        const_dict['value-type constraint'] = properties_with_data_constraints[prop]['value-type constraint']
    
    if len(const_dict) != 0:
        prop2constraint[prop] = const_dict

In [21]:
len(prop2constraint)

949

In [22]:
prop2constraint

{'P2673': {'subject type constraint': {'P2308': ['Q12280',
    'Q44377',
    'Q12323',
    'Q18984099'],
   'P2309': ['Q21503252']},
  'value-type constraint': {'P2308': ['Q12280',
    'Q44377',
    'Q12323',
    'Q18984099'],
   'P2309': ['Q21503252']}},
 'P186': {'subject type constraint': {'P2308': ['Q223557',
    'Q202866',
    'Q15706911',
    'Q838948',
    'Q811979',
    'Q66661745',
    'Q35758',
    'Q214609',
    'Q19861951',
    'Q10683158',
    'Q16686448'],
   'P2309': ['Q30208840'],
   'P2303': ['Q186422']},
  'value-type constraint': {'P2308': ['Q214609',
    'Q2095',
    'Q11344',
    'Q4936952',
    'Q25403900',
    'Q1362373',
    'Q7239',
    'Q40050',
    'Q223557',
    'Q15053464',
    'Q206615',
    'Q37756',
    'Q11173',
    'Q31839438',
    'Q10683158',
    'Q19829125'],
   'P2309': ['Q30208840'],
   'P2303': ['Q113796']}},
 'P1066': {'subject type constraint': {'P2308': ['Q5', 'Q95074', 'Q35120'],
   'P2309': ['Q21503252']},
  'value-type constraint': {'P2308'

In [29]:
value_rel_dict = {}

for prop in prop2constraint:
    temp_dict = {}
    for key in prop2constraint[prop]:
        temp_dict[key] = {"P2309": prop2constraint[prop][key]["P2309"], "P2308": prop2constraint[prop][key]["P2308"]}

    value_rel_dict[prop] = temp_dict
             
len(value_rel_dict)

949

In [30]:
value_rel_dict

{'P2673': {'subject type constraint': {'P2309': ['Q21503252'],
   'P2308': ['Q12280', 'Q44377', 'Q12323', 'Q18984099']},
  'value-type constraint': {'P2309': ['Q21503252'],
   'P2308': ['Q12280', 'Q44377', 'Q12323', 'Q18984099']}},
 'P186': {'subject type constraint': {'P2309': ['Q30208840'],
   'P2308': ['Q223557',
    'Q202866',
    'Q15706911',
    'Q838948',
    'Q811979',
    'Q66661745',
    'Q35758',
    'Q214609',
    'Q19861951',
    'Q10683158',
    'Q16686448']},
  'value-type constraint': {'P2309': ['Q30208840'],
   'P2308': ['Q214609',
    'Q2095',
    'Q11344',
    'Q4936952',
    'Q25403900',
    'Q1362373',
    'Q7239',
    'Q40050',
    'Q223557',
    'Q15053464',
    'Q206615',
    'Q37756',
    'Q11173',
    'Q31839438',
    'Q10683158',
    'Q19829125']}},
 'P1066': {'subject type constraint': {'P2309': ['Q21503252'],
   'P2308': ['Q5', 'Q95074', 'Q35120']},
  'value-type constraint': {'P2309': ['Q21503252'],
   'P2308': ['Q5', 'Q95074', 'Q16334295', 'Q21070568', 'Q

In [31]:
with open('subject_object_constraints.json', 'w') as f:
    json.dump(value_rel_dict, f)

In [37]:


entity_id = "Q56418902"  # Wikidata entity ID for Barack Obama
relation_id = "P31"  # Relation ID for country of citizenship

connected_objects = get_connected_entities(entity_id, relation_id)

connected_objects

[{'object': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5'}, 'objectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'human'}}]


['Q5']

In [4]:
with open('subject_object_constraints.json', 'r') as f:
    value_rel_dict = json.load(f)

In [13]:
c = Counter()

for prop in value_rel_dict:

    for key in value_rel_dict[prop]:
        for item in value_rel_dict[prop][key]["P2309"]:
            c[item] += 1

c

Counter({'Q21503252': 958, 'Q30208840': 464, 'Q21514624': 65})